# Lab: S3 and MapReduce

We'll write mapper and reducer programs in Python to do a wordcount caculation using multiprocessing. The (optional)
extension applies this to a piece of text hosted in your Amazon S3 bucket. The initial parts of the lab will show you how to
create an Amazon S3 bucket, and once this is accomplished, we'll go onto the Python programs.

## AWS Academy Data Analytics Lab 1

The instructions for this lab are on the AWS Academy canvas page. Follow the link to Modules and then Lab 1.
--> 1. This lab shows you how to store data in Amazon S3. It should take you no more than 30 minutes.

In [0]:
# Finished on 10/Mar/2021

## Creating MapReduce functions using Python

We will write Python (not pyspark) code which returns the frequency of words within a given piece of text. As seen in the
MapReduce lecture and demo, we need to create two Python files, a mapper and a reducer, for Hadoop jobs. The mapper
will split a chunk of text on white space and count the number of occurrences of a word in the chunk. The reducer will
combine the mapped word counts into a single value for each word. Keep in mind that each reducer task is guaranteed to
see all values for the same key, but may also see multiple keys. In this lab, we'll only run the code on the Databricks driver
node.

## MapReduce in Python

This builds on the demo, the code for which is available from Blackboard as a Databricks notebook. You'll implement a
simple version of the mapper and reducer functions for wordcount. Note that the functions are written in Python (not
pyspark) and therefore do not use the SparkContext.

--> 2. Define a variable text which contains some text. Feel free to include punctuation, extra spaces etc.

In [0]:
dummy_text = "    This is line 1\nThis is line 2\nThis is line 3\nThis is line 4\nUh oh, this is the end of the Dummy_text variable   "
print(dummy_text)

This is line 1
This is line 2
This is line 3
This is line 4
Uh oh, this is the end of the Dummy_text variable

--> 3. Find a Python method which removes any white space from the start and the end of an input string. Test that it
works as expected by applying it to your text string (you may need to adjust its contents so you can find out whether
your application of the method works).

In [0]:
dummy_text = dummy_text.strip()
print(dummy_text)

This is line 1
This is line 2
This is line 3
This is line 4
Uh oh, this is the end of the Dummy_text variable

--> 4. Divide the text cleaned up in Point 3 into a list of individual words.

In [0]:
dummy_text = dummy_text.replace(",", " ,")
dummy_text = dummy_text.replace("\n", " ")      # Necessary to remove linebreaks
dummy_list = dummy_text.split(" ")
print(dummy_list)

['This', 'is', 'line', '1', 'This', 'is', 'line', '2', 'This', 'is', 'line', '3', 'This', 'is', 'line', '4', 'Uh', 'oh', ',', 'this', 'is', 'the', 'end', 'of', 'the', 'Dummy_text', 'variable']

--> 5. Turn the list of words into a dictionary, where the key is each word and the value is its frequency in the text output
by Point 4. You should use Counter for this purpose. An example of use is below:

In [0]:
from collections import Counter

dummy_dict = Counter(dummy_list)
print(dummy_dict)

Counter({'is': 5, 'This': 4, 'line': 4, 'the': 2, '1': 1, '2': 1, '3': 1, '4': 1, 'Uh': 1, 'oh': 1, ',': 1, 'this': 1, 'end': 1, 'of': 1, 'Dummy_text': 1, 'variable': 1})

--> 6. Use the points above to define the mapper function: this function should take a piece of text as an argument:

In [0]:
def mapper(text):
  text = text.strip()
  text = text.lower()
  text = text.replace(",", " ,")
  text = text.replace("\n", " ")
  list = text.split(" ")
  d = Counter(list)
  return d

In [0]:
#and it should return a Counter dictionary of words with their frequencies in text.
mapper(dummy_text)

Out[7]: Counter({'this': 5,
 'is': 5,
 'line': 4,
 '1': 1,
 '2': 1,
 '3': 1,
 '4': 1,
 'uh': 1,
 'oh': 1,
 '': 1,
 ',': 1,
 'the': 2,
 'end': 1,
 'of': 1,
 'dummy_text': 1,
 'variable': 1})

--> 7. Now for the reducer function. This should take two arguments, both Counter dictionaries and use the update method
to update the values in cnt1 with the values in cnt2. It should return the updated Counter dictionary. I.e., it's header
should look like:

In [0]:
# Dictionary - update() method
def reducer(cnt1, cnt2):
  cnt1.update(cnt2)
  return cnt1

--> 8. You can use readlines to read the contents of a file into a list.

(a) Copy your pride and prejudice.txt file from the /FileStore/ to your local /tmp directory.

In [0]:
dbutils.fs.cp("dbfs:/FileStore/pride_and_prejudice.txt", "file:/tmp/tables/pride_and_prejudice.txt")

Out[9]: True

In [0]:
dbutils.fs.ls("file:/tmp/tables/")

Out[10]: [FileInfo(path='file:/tmp/tables/pride_and_prejudice.txt', name='pride_and_prejudice.txt', size=799645)]

In [0]:
pnp_file = "file:/tmp/tables/pride_and_prejudice.txt"
dbutils.fs.head(pnp_file, 1000)

[Truncated to first 1000 bytes]
Out[11]: '\ufeffThe Project Gutenberg eBook of Pride and Prejudice, by Jane Austen\r\n\r\nThis eBook is for the use of anyone anywhere in the United States and\r\nmost other parts of the world at no cost and with almost no restrictions\r\nwhatsoever. You may copy it, give it away or re-use it under the terms\r\nof the Project Gutenberg License included with this eBook or online at\r\nwww.gutenberg.org. If you are not located in the United States, you\r\nwill have to check the laws of the country where you are located before\r\nusing this eBook.\r\n\r\nTitle: Pride and Prejudice\r\n\r\nAuthor: Jane Austen\r\n\r\nRelease Date: June, 1998 [eBook #1342]\r\n[Most recently updated: February 10, 2021]\r\n\r\nLanguage: English\r\n\r\nCharacter set encoding: UTF-8\r\n\r\nProduced by: Anonymous Volunteers and David Widger\r\n\r\n*** START OF THE PROJECT GUTENBERG EBOOK PRIDE AND PREJUDICE ***\r\n\r\nTHERE IS AN ILLUSTRATED EDITION OF THIS TITLE WHICH MAY VIEWED AT EBOOK\r\n[# 42671 ]\r\n\r\ncover\r\n\r\n\r\n\r\n\r\n Pride and Prejudice\r\n\r\n By Jane Austen\r\n\r\n '

(b) Modify the following code to read the local file you've just created into a list called data

In [0]:
try :
  file = open("/tmp/tables/pride_and_prejudice.txt", "r")
  data = file.readlines()
finally :
  file.close()

--> 9. Next you can copy and paste the chunks and the chunk mapper functions which dived the input into chunks and run
the mapper and reducer over each chunk. If you have a compatibility issue (i.e. mapper or reducer expect more /
fewer inputs) when you try to run the chunk mapper, double check that all mapper and reducer arguments are as
expected.

In [0]:
from math import ceil
from functools import reduce

def chunks(l, n):
# """ Yield n successive chunks from l."""
  list_len = len(l)
  chunk_length = ceil(list_len/n)
  for i in range(0, list_len, chunk_length):
    yield l[i:i + chunk_length]

def chunk_mapper(chunk):
  mapped = map(mapper, chunk)
  reduced = reduce(reducer, mapped)
  return reduced

--> 10. Copy the pride and prejudice.txt le we created for one of the homeworks from the /FileStore to local /tmp/.

In [0]:
# Already done

--> 11. Read its contents into a variable called data using readlines.

In [0]:
# Already done above

print(data)

['\ufeffThe Project Gutenberg eBook of Pride and Prejudice, by Jane Austen\n', '\n', 'This eBook is for the use of anyone anywhere in the United States and\n', 'most other parts of the world at no cost and with almost no restrictions\n', 'whatsoever. You may copy it, give it away or re-use it under the terms\n', 'of the Project Gutenberg License included with this eBook or online at\n', 'www.gutenberg.org. If you are not located in the United States, you\n', 'will have to check the laws of the country where you are located before\n', 'using this eBook.\n', '\n', 'Title: Pride and Prejudice\n', '\n', 'Author: Jane Austen\n', '\n', 'Release Date: June, 1998 [eBook #1342]\n', '[Most recently updated: February 10, 2021]\n', '\n', 'Language: English\n', '\n', 'Character set encoding: UTF-8\n', '\n', 'Produced by: Anonymous Volunteers and David Widger\n', '\n', '*** START OF THE PROJECT GUTENBERG EBOOK PRIDE AND PREJUDICE ***\n', '\n', 'THERE IS AN ILLUSTRATED EDITION OF THIS TITLE WHICH MAY VIEWED AT EBOOK\n', '[# 42671 ]\n', '\n', 'cover\n', '\n', '\n', '\n', '\n', ' Pride and Prejudice\n', '\n', ' By Jane Austen\n', '\n', ' CONTENTS\n', '\n', ' Chapter 1\n', '\n', ' Chapter 2\n', '\n', ' Chapter 3\n', '\n', ' Chapter 4\n', '\n', ' Chapter 5\n', '\n', ' Chapter 6\n', '\n', ' Chapter 7\n', '\n', ' Chapter 8\n', '\n', ' Chapter 9\n', '\n', ' Chapter 10\n', '\n', ' Chapter 11\n', '\n', ' Chapter 12\n', '\n', ' Chapter 13\n', '\n', ' Chapter 14\n', '\n', ' Chapter 15\n', '\n', ' Chapter 16\n', '\n', ' Chapter 17\n', '\n', ' Chapter 18\n', '\n', ' Chapter 19\n', '\n', ' Chapter 20\n', '\n', ' Chapter 21\n', '\n', ' Chapter 22\n', '\n', ' Chapter 23\n', '\n', ' Chapter 24\n', '\n', ' Chapter 25\n', '\n', ' Chapter 26\n', '\n', ' Chapter 27\n', '\n', ' Chapter 28\n', '\n', ' Chapter 29\n', '\n', ' Chapter 30\n', '\n', ' Chapter 31\n', '\n', ' Chapter 32\n', '\n', ' Chapter 33\n', '\n', ' Chapter 34\n', '\n', ' Chapter 35\n', '\n', ' Chapter 36\n', '\n', ' Chapter 37\n', '\n', ' Chapter 38\n', '\n', ' Chapter 39\n', '\n', ' Chapter 40\n', '\n', ' Chapter 41\n', '\n', ' Chapter 42\n', '\n', ' Chapter 43\n', '\n', ' Chapter 44\n', '\n', ' Chapter 45\n', '\n', ' Chapter 46\n', '\n', ' Chapter 47\n', '\n', ' Chapter 48\n', '\n', ' Chapter 49\n', '\n', ' Chapter 50\n', '\n', ' Chapter 51\n', '\n', ' Chapter 52\n', '\n', ' Chapter 53\n', '\n', ' Chapter 54\n', '\n', ' Chapter 55\n', '\n', ' Chapter 56\n', '\n', ' Chapter 57\n', '\n', ' Chapter 58\n', '\n', ' Chapter 59\n', '\n', ' Chapter 60\n', '\n', ' Chapter 61\n', '\n', '\n', '\n', '\n', ' Chapter 1\n', '\n', ' It is a truth universally acknowledged, that a single man in\n', ' possession of a good fortune, must be in want of a wife.\n', '\n', ' However little known the feelings or views of such a man may be\n', ' on his first entering a neighbourhood, this truth is so well\n', ' fixed in the minds of the surrounding families, that he is\n', ' considered the rightful property of some one or other of their\n', ' daughters.\n', '\n', ' “My dear Mr. Bennet,” said his lady to him one day, “have you\n', ' heard that Netherfield Park is let at last?”\n', '\n', ' Mr. Bennet replied that he had not.\n', '\n', ' “But it is,” returned she; “for Mrs. Long has just been here, and\n', ' she told me all about it.”\n', '\n', ' Mr. Bennet made no answer.\n', '\n', ' “Do you not want to know who has taken it?” cried his wife\n', ' impatiently.\n', '\n', ' “_You_ want to tell me, and I have no objection to hearing it.”\n', '\n', ' This was invitation enough.\n', '\n', ' “Why, my dear, you must know, Mrs. Long says that Netherfield is\n', ' taken by a young man of large fortune from the north of England;\n', ' that he came down on Monday in a chaise and four to see the\n', ' place, and was so much delighted with it, that he agreed with Mr.\n', ' Morris immediately; that he is to take possession before\n', ' Michaelmas, and some of his servants are to be in the house by\n', ' the end of next week.”\n', '\n', ' 

--> 12. Set up data chunks so it divides your data into 32 chunks.

In [0]:
data_chunks = chunks(data, 32)

--> 13. Now you can use the multiprocessing to run your code in parallel!

In [0]:
from multiprocessing import Pool

pool = Pool(8)
# step 1:
mapped = pool.map(chunk_mapper ,data_chunks )
# step 2:
reduced = reduce(reducer , mapped)
print(reduced)

Counter({',': 8852, 'the': 4492, 'to': 4180, 'of': 3696, 'and': 3503, '': 2425, 'her': 2054, 'a': 1981, 'in': 1904, 'was': 1817, 'i': 1732, 'she': 1652, 'that': 1477, 'not': 1399, 'he': 1285, 'it': 1282, 'his': 1241, 'be': 1230, 'you': 1209, 'as': 1174, 'had': 1158, 'with': 1075, 'for': 1039, 'but': 894, 'is': 845, 'have': 834, 'at': 793, 'mr.': 766, 'on': 700, 'by': 650, 'my': 647, 'him': 604, 'they': 585, 'all': 572, 'were': 557, 'so': 551, 'which': 538, 'elizabeth': 537, 'could': 513, 'been': 507, 'from': 497, 'very': 474, 'would': 462, 'no': 459, 'their': 437, 'this': 435, 'what': 431, 'your': 425, ',”': 411, 'will': 406, 'such': 388, 'said': 373, 'or': 368, 'an': 356, 'when': 354, 'are': 343, 'me': 340, 'mrs.': 338, 'them': 330, 'if': 323, 'darcy': 317, 'must': 313, 'do': 313, 'there': 310, 'am': 308, 'more': 303, 'much': 302, 'any': 301, '“i': 295, 'than': 283, 'who': 282, 'miss': 272, 'bennet': 266, 'one': 250, 'should': 244, 'did': 238, 'we': 236, 'jane': 229, 'has': 217, 'know': 217, 'though': 216, 'never': 215, 'bingley': 213, 'only': 212, 'soon': 206, 'some': 204, 'can': 204, 'other': 202, 'how': 202, 'think': 201, 'may': 200, 'before': 198, 'might': 198, 'after': 197, 'every': 197, 'most': 188, 'now': 185, 'little': 177, 'herself': 177, 'time': 177, 'without': 176, 'lady': 174, 'being': 173, 'own': 167, 'make': 166, 'good': 163, 'nothing': 161, 'then': 160, 'shall': 159, 'well': 156, 'see': 149, 'dear': 146, 'collins': 143, 'sister': 142, 'into': 141, 'first': 138, 'say': 133, 'great': 133, 'however': 132, 'too': 130, 'again': 128, 'young': 127, 'two': 126, 'wickham': 124, 'give': 122, 'chapter': 122, 'made': 122, 'hope': 121, 'about': 120, 'many': 119, 'always': 118, 'ever': 116, 'up': 116, 'man': 114, 'cannot': 113, 'out': 113, 'family': 112, 'thought': 109, 'lydia': 108, 'its': 107, 'it.': 107, 'day': 105, 'last': 105, 'father': 104, 'long': 102, 'catherine': 102, 'saw': 100, 'us': 97, 'mother': 96, 'quite': 96, 'felt': 96, '“and': 95, 'wish': 95, 'letter': 95, 'replied': 94, 'having': 94, '“you': 93, 'till': 92, 'where': 91, 'himself': 91, 'sure': 89, 'her.': 89, 'really': 87, 'often': 85, 'whom': 85, 'house': 84, 'go': 84, 'project': 83, 'away': 83, 'cried': 83, 'way': 83, 'enough': 83, 'even': 83, '“but': 82, 'came': 82, 'believe': 82, 'manner': 80, 'come': 79, 'pleasure': 79, 'heard': 78, 'take': 78, 'feelings': 77, 'over': 77, 'does': 77, 'our': 76, 'love': 76, 'less': 76, 'friend': 76, 'whole': 76, 'room': 75, 'like': 74, 'gardiner': 74, 'therefore': 73, 'looked': 73, 'few': 72, 'him.': 72, 'while': 72, 'done': 72, 'upon': 71, 'yet': 71, 'still': 70, 'anything': 70, 'nor': 70, 'myself': 69, 'just': 68, 'told': 68, 'down': 68, 'better': 68, 'something': 68, 'ladies': 68, 'longbourn': 68, 'next': 67, 'towards': 67, 'seen': 67, 'found': 67, 'aunt': 67, 'happy': 66, 'went': 66, 'let': 65, 'tell': 65, 'hear': 65, 'left': 65, 'subject': 65, 'seemed': 65, 'same': 65, 'perhaps': 64, 'morning': 64, 'colonel': 64, 'certainly': 63, 'sir': 63, 'these': 63, 'part': 63, 'received': 63, 'each': 63, 'between': 62, 'charlotte': 62, 'almost': 61, 'speak': 61, 'present': 61, 'added': 61, 'both': 61, '“it': 60, 'seeing': 60, 'least': 60, 'place': 59, 'lizzy': 59, 'those': 59, '“that': 59, 'whose': 59, 'leave': 59, 'work': 59, 'netherfield': 58, 'opinion': 58, 'sisters': 58, 'passed': 58, 'evening': 57, 'daughter': 57, 'kitty': 57, 'knew': 57, 'because': 57, 'lucas': 56, 'since': 56, 'find': 56, 'going': 56, 'coming': 56, 'gone': 55, 'rather': 55, 'given': 55, 'happiness': 55, 'three': 55, 'gutenberg-tm': 55, 'known': 54, 'gave': 54, 'means': 54, 'able': 54, 'far': 54, 'them.': 53, 'began': 53, 'either': 53, 'against': 53, 'behaviour': 53, 'brother': 53, 'look': 53, 'took': 53, 'indeed': 52, 'here': 51, 'general': 51, 'conversation': 51, 'moment': 51, 'whether': 51, 'kind': 51, 'marriage': 51, 'uncle': 51, '“my': 50, 'get': 50, 'attention': 50, 'off': 50, 'affection': 50, 'bingley’s': 49, 'another': 49, 'immediately': 49, 'cha

## Optional: run your MapReduce code on data you have stored in your S3 bucket
### AWS Educate: Create a (lasting) S3 bucket

The AWS Academy environment is set up for you, which is very helpful for getting everything started up quickly, and it
means that it all gets torn down at the end of the lab so you don't need to worry about running out of credits, but there
are various rewall rules applied which mean you can't always do everything you may have wanted with your resources. In
this section, you will create an S3 bucket that you can use from other programs, though obviously you will pay (in credits
in your AWS Educate account; it will be within the free tier limits in AWS Free Tier) for storage of any objects you upload,
download or store in there. You will get to create an S3 bucket next week as well, so you will be able to practise these skills
again.

--> 14. Log into your AWS Educate account and go to classroom (BDTT 2020-2021 { you may have to follow a new invite).

--> 15. Click on AWS console (you may need to click on Continue rst and allow pop ups from this source).

--> 16. Under AWS services, nd and select S3.

--> 17. The rst time you use this service, the bucket overview page will show no buckets. Click Create bucket.

--> 18. Enter Bucket name: follow the rules for naming. Ensure that you make a note of your bucket name.

--> 19. Leave AWS region set to us-east-1 (only us-east-1, Northern Virginia, is supported by AWS Educate Account).

--> 20. Leave all other selections, including \block all public access" as they are.

--> 21. Click Create bucket. This has now created your bucket and will return you to the Buckets summary page..

--> 22. Select the bucket you have just created from the list of buckets on the Buckets summary page.

--> 23. You will see there are no objects in it. Click upload to upload your data. Remember to organize your storage well
(i.e. consider using folders when that would make your life easier!) Select the le you wish to upload from your local
machine.

--> 24. Leave other settings on defaults. This includes the storage class (standard access) and access control lists giving only
you permission to read.

--> 25. Download the ebook (Alice's Adventures in Wonderland) from https://www.gutenberg.org/files/11/11-0.txt to
your local machine.

--> 26. Click on upload, select the le you've just downloaded and wait for the upload to nish (the status will change to
Suceeded).

--> 27. You can view the contents of the bucket by clicking on the bucket name from the Buckets summary page.

## Read the data from your S3 bucket in your Python notebook
The following Python code will read the file -my key- from the bucket -my bucket- in your Python notebook.

In [0]:
# importing the boto3 library
import boto3
from botocore import UNSIGNED
from botocore.client import Config

# connect to S3 using boto3 client
s3_client = boto3.client(service_name ='s3', config = Config(signature_version = UNSIGNED))

# # get S3 object
result = s3_client.get_object(Bucket = 'ambyaws', Key = 'pride_and_prejudice.txt')

# # Read a text file line by line using splitlines object
for line in result["Body"].read().splitlines():
  each_line = line.decode('utf-8')
  print(each_line)

The Project Gutenberg EBook of Alice’s Adventures in Wonderland, by Lewis Carroll

This eBook is for the use of anyone anywhere in the United States and most
other parts of the world at no cost and with almost no restrictions
whatsoever. You may copy it, give it away or re-use it under the terms of
the Project Gutenberg License included with this eBook or online at
www.gutenberg.org. If you are not located in the United States, you'll have
to check the laws of the country where you are located before using this ebook.

Title: Alice’s Adventures in Wonderland

Author: Lewis Carroll

Release Date: June 25, 2008 [EBook #11]
[Most recently updated: October 12, 2020]

Language: English

Character set encoding: UTF-8

*** START OF THIS PROJECT GUTENBERG EBOOK ALICE’S ADVENTURES IN WONDERLAND ***



Produced by Arthur DiBianca and David Widger

[Illustration]




Alice’s Adventures in Wonderland

by Lewis Carroll

THE MILLENNIUM FULCRUM EDITION 3.0

Contents

 CHAPTER I. Down the Rabbit-Hole
 CHAPTER II. The Pool of Tears
 CHAPTER III. A Caucus-Race and a Long Tale
 CHAPTER IV. The Rabbit Sends in a Little Bill
 CHAPTER V. Advice from a Caterpillar
 CHAPTER VI. Pig and Pepper
 CHAPTER VII. A Mad Tea-Party
 CHAPTER VIII. The Queen’s Croquet-Ground
 CHAPTER IX. The Mock Turtle’s Story
 CHAPTER X. The Lobster Quadrille
 CHAPTER XI. Who Stole the Tarts?
 CHAPTER XII. Alice’s Evidence




CHAPTER I.
Down the Rabbit-Hole


Alice was beginning to get very tired of sitting by her sister on the
bank, and of having nothing to do: once or twice she had peeped into
the book her sister was reading, but it had no pictures or
conversations in it, “and what is the use of a book,” thought Alice
“without pictures or conversations?”

So she was considering in her own mind (as well as she could, for the
hot day made her feel very sleepy and stupid), whether the pleasure of
making a daisy-chain would be worth the trouble of getting up and
picking the daisies, when suddenly a White Rabbit with pink eyes ran
close by her.

There was nothing so _very_ remarkable in that; nor did Alice think it
so _very_ much out of the way to hear the Rabbit say to itself, “Oh
dear! Oh dear! I shall be late!” (when she thought it over afterwards,
it occurred to her that she ought to have wondered at this, but at the
time it all seemed quite natural); but when the Rabbit actually _took a
watch out of its waistcoat-pocket_, and looked at it, and then hurried
on, Alice started to her feet, for it flashed across her mind that she
had never before seen a rabbit with either a waistcoat-pocket, or a
watch to take out of it, and burning with curiosity, she ran across the
field after it, and fortunately was just in time to see it pop down a
large rabbit-hole under the hedge.

In another moment down went Alice after it, never once considering how
in the world she was to get out again.

The rabbit-hole went straight on like a tunnel for some way, and then
dipped suddenly down, so suddenly that Alice had not a moment to think
about stopping herself before she found herself falling down a very
deep well.

Either the well was very deep, or she fell very slowly, for she had
plenty of time as she went down to look about her and to wonder what
was going to happen next. First, she tried to look down and make out
what she was coming to, but it was too dark to see anything; then she
looked at the sides of the well, and noticed that they were filled with
cupboards and book-shelves; here and there she saw maps and pictures
hung upon pegs. She took down a jar from one of the shelves as she
passed; it was labelled “ORANGE MARMALADE”, but to her great
disappointment it was empty: she did not like to drop the jar for fear
of killing somebody underneath, so managed to put it into one of the
cupboards as she fell past it.

“Well!” thought Alice to herself, “after such a fall as this, I shall
think nothing of tumbling down stairs! How brave they’ll all think me
at home! Why, I wouldn’t say anything abo

--> If you run this (even after replacing <my bucket> and <my key> with your values) on Databricks Community Edition,
it'll fail due to Access denied. The Databricks Community Edition doesn't appear to allow you to store your credentials and
therefore to read from your own S3 bucket, you'll need to make the object public. Warning: You need to ensure that
you do not leave the object with public permissions after the lab is over.
  
--> In order to make the above script work, S3 bucket should be accessed by everyone (public) and objects individually also should be accessed by everyone (public).

--> 28. Click on the bucket name.

--> 29. Select the Permissions tab.

--> 30. Click Edit under Block public access.

--> 31. Untick Block all public access and click on Save changes.

--> 32. Type conrm in the Edit Block public access box and click conrm.

--> 33. Return to the bucket and select the object you want to make public and choose Make public under Actions.

Now you can read your data in using the boto3 code above and it'll work! You should run the MapReduce program now
using the second part of this lab.

Stop public access to your objects
Even if you don't delete them, you need to stop the objects in your bucket being public. If you leave the items
public and someone nds them, they could run your account out of credit without your knowledge.

--> 33. Navigate to the bucket.

--> 34. Click on Permissions.

--> 35. Click Edit under Block public access.

--> 36. Tick Block all public access and click on Save changes.

--> 37. Type conrm in the Edit Block public access box and click conrm.

## Delete your S3 bucket
Keeping data in S3 costs money (or credits), so unless you plan on reusing the data in Standard storage, you should delete
the bucket.

--> 38. Click the radio button next to the bucket you want to delete and click on Delete.

--> 39. If the bucket has contents, you will need to empty it rst { the UI guides you through the steps for this.

(a) Click on the link in the box that's appeared containing Buckets must be empty before they can be deleted.
To delete all objects in the bucket, use the empty bucket conguration.

(b) Type the required phrase in the conrm box and click Empty.

--> 40. Return to the Bucket summary page, select bucket and click on Delete.

--> 41. Type the required phase in the conrm box and click Delete.

You should now not be incurring any charges in your AWS Educate account.